In [1]:
!pip install ortools

In [5]:
import pandas as pd
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp

In [13]:
distance_matrix=pd.read_excel(open('P4_VRPtraveltimes.xlsx', 'rb'),
              sheet_name='distancematrix')
distance_matrix.head()

,Unnamed: 0,T4X 0B6,T5A 5C1,T5B 0S1,T5E 4C6,T5J 1V7,T5J 3N3,T5K 0M8,T5L 4Z6,T5M 2L7,T5M 3L7,T5T 1K8,T5T 5L5,T5Y 0L2,T5Z 3J8,T6B 0P2,T6E 2A3,T6E 5A7,T6H 5T1,T6J 6P9,T6K 4B4,T6R 0G4,T6T 0C2,T6V 1J6,T6W 1A2,T6X 0P2,T7X 2K6,T7Z 2W7,T8A 4N5,T8A 5V9,T8H 1S8,T8L 3W2,T8N 3L3,T8N 5T8,T8R 1R4,T9E 6Z7
0,T4X 0B6,0,33,34,34,35,36,37,40,41,38,28,35,33,37,28,29,28,24,23,18,26,23,36,19,18,41,48,24,29,26,45,45,39,54,20
1,T5A 5C1,29,0,14,14,22,24,24,20,21,22,28,29,11,13,19,27,24,28,30,26,33,19,15,26,27,37,42,15,17,14,25,27,22,33,36
2,T5B 0S1,31,15,0,7,10,13,16,15,13,14,22,21,15,10,11,16,16,25,30,24,31,21,18,28,29,31,35,17,18,15,31,27,22,36,38
3,T5E 4C6,31,16,8,0,14,14,14,9,11,12,20,19,15,8,15,21,19,28,31,28,30,21,12,28,29,29,33,17,19,16,30,24,19,30,38
4,T5J 1V7,36,25,13,13,0,8,10,22,14,17,30,23,25,20,16,15,20,23,29,29,30,28,25,28,29,38,42,23,29,23,41,35,29,43,37


In [16]:
#Create a list of lists
distance=distance_matrix.iloc[:,1:].values.tolist()

In [18]:
"""Simple Vehicles Routing Problem (VRP)"""

def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['distance_matrix'] = distance
    data['num_vehicles'] = 6
    data['depot'] = 2
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    max_route_distance = 0
    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n'.format(vehicle_id)
        route_distance = 0
        while not routing.IsEnd(index):
            plan_output += ' {} -> '.format(manager.IndexToNode(index))
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
        plan_output += '{}\n'.format(manager.IndexToNode(index))
        plan_output += 'Distance of the route: {}m\n'.format(route_distance)
        print(plan_output)
        max_route_distance = max(route_distance, max_route_distance)
    print('Maximum of the route distances: {}m'.format(max_route_distance))



def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        3000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100)

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)
    else:
        print('No solution found !')


if __name__ == '__main__':
    main()

Objective: 10359
Route for vehicle 0:
 2 ->  13 ->  33 ->  31 ->  32 ->  22 ->  7 ->  3 -> 2
Distance of the route: 94m

Route for vehicle 1:
 2 ->  27 ->  29 ->  28 ->  30 ->  12 ->  1 -> 2
Distance of the route: 98m

Route for vehicle 2:
 2 ->  4 ->  6 ->  5 ->  15 ->  17 ->  16 ->  14 -> 2
Distance of the route: 75m

Route for vehicle 3:
 2 ->  8 ->  9 ->  20 ->  18 ->  23 ->  24 ->  19 -> 2
Distance of the route: 97m

Route for vehicle 4:
 2 ->  0 ->  34 ->  21 -> 2
Distance of the route: 98m

Route for vehicle 5:
 2 ->  26 ->  25 ->  11 ->  10 -> 2
Distance of the route: 97m

Maximum of the route distances: 98m
